In [2]:
import sys
sys.path.append("./functions")

from utils import *
from model_bet_varying_layers import *
from layer import *
import pandas as pd

In [4]:
param = { 
    "graph_files": ["ER_15_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_15_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],

    "generation_seeds": [10],
    "num_train" : 5,
    "num_test" : 10,
    "num_copies": [10],#[100],
    "adj_size" : 10000,
    "split_seeds": [10],

    'model_seed' : 15,
    "emb_dimensions": [2,4,8,12,16,20],
    "num_layers": [1,4,7],
    "num_epochs": 15,    

}

results = {'g': [], 'layers':[],'emb':[], 'epoch':[], 'kt':[], 'std':[]}
for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:

        print(graph_file,emb,num_layers)
        data_path_test = f"{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle"
        #Load test data
        with open("./data_splits/test/"+data_path_test,"rb") as fopen:
            list_graph_test,list_n_seq_test,list_num_node_test,bc_mat_test,deg_mat_test = pickle.load(fopen)    

        for num_copies in param["num_copies"]:
            for splitseed in param["split_seeds"]:

                data_path_train = f"{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle"

                #Load training data
                print(f"Loading data...")
                with open("./data_splits/train/"+data_path_train,"rb") as fopen:
                    list_graph_train,list_n_seq_train,list_num_node_train,bc_mat_train,deg_mat_train = pickle.load(fopen)

                model_size = bc_mat_train.shape[0]
                assert model_size == param["adj_size"]

                for emb in param["emb_dimensions"]:
                    for num_layers in param["num_layers"]:
                        
                        #Get adjacency matrices from graphs
                        print(f"Graphs to adjacency conversion.")

                        list_adj_train,list_adj_t_train = graph_to_adj_bet(list_graph_train,list_n_seq_train,list_num_node_train,model_size)
                        list_adj_test,list_adj_t_test = graph_to_adj_bet(list_graph_test,list_n_seq_test,list_num_node_test,model_size)

                        #Model parameters
                        torch.manual_seed(param["model_seed"])

                        hidden = emb

                        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

                        print(f"GNN_Bet{num_layers}")
                        exec(f'model = GNN_Bet{num_layers}(ninput=model_size,nhid=hidden,dropout=0.6)')
                        model.to(device)

                        optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
                        num_epoch = param["num_epochs"]

                        print("Training")
                        print(f"Total Number of epoches: {num_epoch}")

                        for e in range(num_epoch):
                            print(f"Epoch number: {e+1}/{num_epoch}")
                            train(list_adj_train,list_adj_t_train,list_num_node_train,bc_mat_train,model=model,device=device,optimizer=optimizer,size=model_size)

                            #to check test loss while training
                            with torch.no_grad():
                                r = test(list_adj_test,list_adj_t_test,list_num_node_test,bc_mat_test,deg_mat_test,model=model,device=device,size=model_size)
                        
                            results['g'].append(data_path_train)
                            results['layers'].append(num_layers)
                            results["emb"].append(emb)
                            results['epoch'].append(e)
                            results['kt'].append(r["kt"])
                            results['std'].append(r["std"])

                            df = pd.DataFrame.from_dict(results)
                            df.to_csv("./outputs/output_ablationtest_varying_embedings.csv")

ER_15_graphs_1000_500_nodes 4 4
Loading data...
Graphs to adjacency conversion.
Processing 50 graphs...
Processing 10 graphs...
GNN_Bet1
Training
Total Number of epoches: 15
Epoch number: 1/15
   Average KT score on test graphs is: 0.1722911275503344 and std: 0.031022336743115705
Epoch number: 2/15
   Average KT score on test graphs is: 0.297094534490158 and std: 0.03368118182142165
Epoch number: 3/15
   Average KT score on test graphs is: 0.6270487646359038 and std: 0.1159313269027083
Epoch number: 4/15
   Average KT score on test graphs is: 0.6284269772330079 and std: 0.11426922609425874
Epoch number: 5/15
   Average KT score on test graphs is: 0.6297476341013092 and std: 0.11497582068570257
Epoch number: 6/15
   Average KT score on test graphs is: 0.6289529322088179 and std: 0.11431650780587854
Epoch number: 7/15
   Average KT score on test graphs is: 0.6286681205203968 and std: 0.11434043381748356
Epoch number: 8/15
   Average KT score on test graphs is: 0.6305851341849592 and std:

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 10})


xs = range(1,8)

# scale free
#title = 'Scale-free'
ys_1 = [0.9734,	0.9752,	0.976,	0.9762,	0.9763,	0.9765,	0.9762]
e_1 =  [0.0014,	0.0017,	0.002,	0.0022,	0.0026,	0.0024,	0.0024]

# erdos renyi
#title = 'Erdos-Renyi'
#
#ys_1 = []
#e_1 =  []
ys_4 = [0.8798,	0.8841,	0.8862,	0.8449,	0.8524,	0.8509,	0.8669]
e_4 =  [0.0401,	0.0358,	0.0364,	0.0714,	0.096,	0.1018,	0.0741]
#ys_7 = [
#e_7 =  []


# GRP
#title = 'Gaussian Random Partition'
#
#ys_1 = []
#e_1 =  []
#ys_4 = []
#e_4 =  []
ys_7 = [0.7485,	0.7913,	0.8573,	0.8419,	0.8349,	0.8278,	0.7603]
e_7 =  [0.227,	0.1625, 0.069,	0.0911,	0.1071,	0.1099,	0.2095]

plt.errorbar(xs[:],ys_1[:],e_1[:])
plt.scatter(xs[:],ys_1[:],c='b')
plt.plot(xs[:],ys_1[:],c='b',label='Scalefree')

plt.errorbar(xs[:],ys_4[:],e_4[:],c='darkred')
plt.scatter(xs[:],ys_4[:],c='darkred')
plt.plot(xs[:],ys_4[:],c='darkred',label='Erdos-Renyi')

plt.errorbar(xs[:],ys_7[:],e_7[:],c='g')
plt.scatter(xs[:],ys_7[:],c='g')
plt.plot(xs[:],ys_7[:],c='g',label='Gaussian Random Partition')

plt.ylabel("KT Score")
plt.xlabel("No. of layers")
plt.xticks(xs,xs)
plt.legend()
#plt.title(title)
plt.ylim([0.6,1])
plt.show()

In [ ]:
import matplotlib.pyplot as plt



xs = [4,8,12,16,20]

# scale free
#title = 'Scale-free'
#ys_1 = [0.9706,	0.9731,	0.9736,	0.9733,	0.9734]
#e_1 =  [0.002,	0.0015,	0.0011,	0.0014,	0.0014]
#ys_4 = [0.9758,	0.9766,	0.9765,	0.9764,	0.9762]
#e_4 =  [0.0018,	0.002,	0.0023,	0.0023,	0.0022]
#ys_7 = [0.9759,	0.9768,	0.9765,	0.9762,	0.9762]
#e_7 =  [0.0023,	0.0023,	0.0022,	0.0023,	0.0024]


# erdos renyi
title = 'Erdos-Renyi'
ys_1 = [0.8577, 0.8724, 0.8786, 0.8812, 0.8798]
e_1 =  [0.0595, 0.0437, 0.0387, 0.0417, 0.0401]
ys_4 = [0.869, 0.8739, 0.8787, 0.8635, 0.8449]
e_4 =  [0.0553, 0.0368, 0.051, 0.0751, 0.0714]
ys_7 = [0.8815, 0.8593, 0.8706, 0.8596, 0.8669]
e_7 =  [0.043, 0.0512, 0.0652, 0.0995, 0.0741]


# GRP
#title = 'Gaussian Random Partition'
#ys_1 = [0.6548	,0.7765	,0.8231	,0.775	,0.7485]
#e_1 =  [0.3383	,0.1815	,0.109	,0.1933	,0.227]
#ys_4 = [0.7252	,0.7747	,0.8346	,0.8196	,0.8419]
#e_4 =  [0.2601	,0.1777	,0.098	,0.116	,0.0911]
#ys_7 = [0.8169	,0.7813	,0.7612	,0.7388	,0.7603]
#e_7 =  [0.1013	,0.1781	,0.21	,0.2542	,0.2095]

plt.errorbar(xs[:],ys_1[:],e_1[:])
plt.scatter(xs[:],ys_1[:],c='b')
plt.plot(xs[:],ys_1[:],c='b',label='1-layered')

plt.errorbar(xs[:],ys_4[:],e_4[:],c='g')
plt.scatter(xs[:],ys_4[:],c='g')
plt.plot(xs[:],ys_4[:],c='g',label='4-layered')

plt.errorbar(xs[:],ys_7[:],e_7[:],c='darkred')
plt.scatter(xs[:],ys_7[:],c='darkred')
plt.plot(xs[:],ys_7[:],c='darkred',label='7-layered')

plt.ylabel("KT Score")
plt.xlabel("Embedding dimensions")
plt.xticks(xs,xs)
plt.legend(loc=4)
plt.title(title)
plt.ylim([0.7,1])
plt.show()